In [2]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

$ H(x) = - \sum_x P(x) log_2 P(x) $

$ IM(x, y) = \sum_x \sum_y P(x, y) log_2 \frac{ P(x, y) }{ P(x)P(y) } $

In [12]:
df = pandas.read_csv('../Datasets/denue_inegi.csv', encoding='ISO-8859-1')

In [49]:
def H(x):
    m = len(x)
    values, counts = np.unique(x, return_counts = True)
    nCategories = len(values)
    probabilities = counts/m
    h = -np.sum(probabilities*np.log2(probabilities))
    hMax = -np.log2(1/nCategories)
    return h, hMax
def IM(x, y):
    allPairs = np.array([(i, j) for i in x for j in y])
    print(allPairs)
    pairs, pairsFrequency = np.unique(allPairs, return_counts = True, axis = 0)
    print(pairs, pairsFrequency)

In [ ]:
print(H(df['municipio'].values))
#IM([1, 2, 3], [4, 5, 6])
IM(df['municipio'].values, df['fecha_alta'].values)

(1.522370914986196, 3.4594316186372973)


In [ ]:
sns.countplot(data = df, x = 'municipio')

In [ ]:
sns.countplot(data = df, y = 'municipio')

In [ ]:
sns.countplot(data = df, y = 'municipio', hue = 'per_ocu')

In [ ]:
sns.countplot(data = df, hue = 'municipio', y = 'per_ocu')

## Filtros

In [ ]:
iUno = df['nombre_act'].values == 'Escuelas de educación primaria del sector privado'
iDos = df['nombre_act'].values == 'Escuelas de educación primaria del sector público'
i = np.logical_or(iUno, iDos)

In [ ]:
sns.countplot(y = df['municipio'].values[i], hue = df['nombre_act'].values[i])

In [ ]:
plt.figure()
plt.pie([10, 3, 4], labels = ['A', 'B', 'C'])


In [ ]:
plt.figure()
plt.subplot(1, 3, 1)
plt.title('Aguascalientes')
iAgs = df['municipio'].values == 'Aguascalientes'
cUno = np.sum(np.logical_and(iUno, iAgs))
cDos = np.sum(np.logical_and(iDos, iAgs))
plt.pie([cUno, cDos], labels=['Privadas', 'Públicas'])
plt.subplot(1, 3, 2)
plt.title('Calvillo')
iCal = df['municipio'].values == 'Calvillo'
cUno = np.sum(np.logical_and(iUno, iCal))
cDos = np.sum(np.logical_and(iDos, iCal))
plt.pie([cUno, cDos], labels=['Privadas', 'Públicas'])
plt.subplot(1, 3, 3)
plt.title('Cosío')
iCos = df['municipio'].values == 'Cosío'
cUno = np.sum(np.logical_and(iUno, iCos))
cDos = np.sum(np.logical_and(iDos, iCos))
plt.pie([cUno, cDos], labels=['Privadas', 'Públicas'])
plt.show()